## Preprocessing

### Load embeddings

In [9]:
import codecs
import numpy as np
from numpy import linalg as LA

def load_embeddings_from_np(filename):
    print('loading ...')
    with codecs.open(filename + '.vocab', 'r', 'utf-8') as f_embed:
        vocab = [line.strip() for line in f_embed]
        
    w2i = {w: i for i, w in enumerate(vocab)}
    wv = np.load(filename + '.wv.npy')

    return vocab, wv, w2i


def normalize(wv):
    
    # normalize vectors
    norms = np.apply_along_axis(LA.norm, 1, wv)
    wv = wv / norms[:, np.newaxis]
    return wv


def load_and_normalize(space, filename, vocab, wv, w2i):
    vocab_muse, wv_muse, w2i_muse = load_embeddings_from_np(filename)
    wv_muse = normalize(wv_muse)
    vocab[space] = vocab_muse 
    wv[space] = wv_muse
    w2i[space] = w2i_muse
    print('done')

In [10]:
vocab = {}
wv = {}
w2i = {}

load_and_normalize('bef', 'baseline', vocab, wv, w2i)
load_and_normalize('aft', 'wiki_cds_hw', vocab, wv, w2i)

loading ...
done
loading ...
done


In [11]:
from tqdm import tqdm

def topK(w, space, k=10):
    
    # extract the word vector for word w
    idx = w2i[space][w]
    vec = wv[space][idx, :]
    
    # compute similarity of w with all words in the vocabulary
    sim = wv[space].dot(vec)
    # sort similarities by descending order
    sort_sim = (sim.argsort())[::-1]

    # choose topK
    best = sort_sim[:(k+1)]

    return [vocab[space][i] for i in best if i!=idx]


def similarity(w1, w2, space):
    i1 = w2i[space][w1]
    i2 = w2i[space][w2]
    vec1 = wv[space][i1, :]
    vec2 = wv[space][i2, :]

    return np.inner(vec1,vec2)



### Restrict vocabulary

In [35]:
import string 


def has_punct(w):
    
    if any([c in string.punctuation for c in w]):
        return True
    return False

def has_digit(w):
    
    if any([c in '0123456789' for c in w]):
        return True
    return False

def limit_vocab(space, exclude = None):
    vocab_limited = []
    for w in tqdm(vocab[space][:1000000]): 
        if w.lower() != w:
            continue
        if len(w) >= 20:
            continue
        if has_digit(w):
            continue
        if '_' in w:
            p = [has_punct(subw) for subw in w.split('_')]
            if not any(p):
                vocab_limited.append(w)
            continue
        if has_punct(w):
            continue
        vocab_limited.append(w)
    
    if exclude:
        vocab_limited = list(set(vocab_limited) - set(exclude))
    
    print("size of vocabulary:", len(vocab_limited))
    
    wv_limited = np.zeros((len(vocab_limited), 400))
    for i,w in enumerate(vocab_limited):
        wv_limited[i,:] = wv[space][w2i[space][w],:]
    
    w2i_limited = {w: i for i, w in enumerate(vocab_limited)}
    
    return vocab_limited, wv_limited, w2i_limited



In [36]:
# create the reduced vocabularies and embeddings before and after, without gendered specific words

import json
with codecs.open('pairs_hispanicwhite.json') as f:
    gender_specific = json.load(f)
with codecs.open('definitional_pairs_race.json') as f:
    definitional_pairs = json.load(f)

exclude_words = []
for pair in definitional_pairs:
    exclude_words.append(pair[0])

exclude_words = list(set(exclude_words).union(set(gender_specific)))

# create spaces of limited vocabulary
vocab['limit_bef'], wv['limit_bef'], w2i['limit_bef'] = limit_vocab('bef', exclude = exclude_words)
vocab['limit_aft'], wv['limit_aft'], w2i['limit_aft'] = limit_vocab('aft', exclude = exclude_words)



100%|██████████| 50000/50000 [00:00<00:00, 431436.41it/s]


size of vocabulary: 47240


100%|██████████| 50000/50000 [00:00<00:00, 387314.30it/s]


size of vocabulary: 46654


### Compute bias-by-projection

In [30]:
# create a dictionary of the bias, before and after

def compute_bias_by_projection(space_to_tag, full_space):
    males = wv[space_to_tag].dot(wv[full_space][w2i[full_space]['nonwhite'],:])
    females = wv[space_to_tag].dot(wv[full_space][w2i[full_space]['white'],:])
    d = {}
    for w,m,f in zip(vocab[space_to_tag], males, females):
        d[w] = m-f
    return d

# compute bias-by-projection before and after debiasing
gender_bias_bef = compute_bias_by_projection('limit_bef', 'bef')
#gender_bias_aft = compute_bias_by_projection('limit_bef', 'bef')
gender_bias_aft = compute_bias_by_projection('limit_aft', 'aft')

print(len(gender_bias_bef))
print(len(gender_bias_aft))


486279
486224


## Experiment 1 (Direct Bias)

In [37]:
# Auxiliary functions for experiments by Caliskan et al.

import scipy
import scipy.misc as misc
import itertools


def s_word(w, A, B, space, all_s_words):
    
    if w in all_s_words:
        return all_s_words[w]
    
    mean_a = []
    mean_b = []
    
    for a in A:
        mean_a.append(similarity(w, a, space))
    for b in B:
        mean_b.append(similarity(w, b, space))
        
    mean_a = sum(mean_a)/float(len(mean_a))
    mean_b = sum(mean_b)/float(len(mean_b))
    
    all_s_words[w] = mean_a - mean_b

    return all_s_words[w]


def s_group(X, Y, A, B, space, all_s_words):
    
    total = 0
    for x in X:
        total += s_word(x, A, B, space, all_s_words)
    for y in Y:
        total -= s_word(y, A, B, space, all_s_words)
        
    return total


def p_value_exhust(X, Y, A, B, space):
    
    if len(X) > 10:
        print('might take too long, use sampled version: p_value')
        return
    
    assert(len(X) == len(Y))
    
    all_s_words = {}
    s_orig = s_group(X, Y, A, B, space, all_s_words) 
    
    union = set(X+Y)
    subset_size = int(len(union)/2)
    
    larger = 0
    total = 0
    for subset in tqdm(set(itertools.combinations(union, subset_size))):
        total += 1
        Xi = list(set(subset))
        Yi = list(union - set(subset))
        if s_group(Xi, Yi, A, B, space, all_s_words) > s_orig:
            larger += 1
    print('num of samples', total)
    return larger/float(total)


def p_value_sample(X, Y, A, B, space):
    
    random.seed(10)
    np.random.seed(10)
    all_s_words = {}
    
    assert(len(X) == len(Y))
    length = len(X)
    
    s_orig = s_group(X, Y, A, B, space, all_s_words) 
    
    num_of_samples = min(1000000, int(scipy.special.comb(length*2,length)*100))
    print('num of samples', num_of_samples)
    larger = 0
    for i in range(num_of_samples):
        permute = np.random.permutation(X+Y)
        Xi = permute[:length]
        Yi = permute[length:]
        if s_group(Xi, Yi, A, B, space, all_s_words) > s_orig:
            larger += 1
    
    return larger/float(num_of_samples)



In [18]:
# Experiment 1(white professions-minority professions)

A = ['adnan','desai','mateo','gonzalo','mira','jeune','oni','selassie']
B = ['james', 'george', 'robert','paul','daniel','arthur','taylor','harry']
C = ['doctor', 'lawyer', 'engineer', 'professor', 'executive', 'management', 'business', 'corporation']
D = ['janitor', 'driver', 'farmer', 'cooker', 'worker', 'plumber', 'painter', 'gardener']

print(p_value_exhust(A, B, C, D, 'aft'))

# Experiment 2(white adjectives-minority adjectives)

E = ['smart', 'educated', 'peaceful', 'safe', 'friend', 'professional','skilled','privileged']
F = ['dumb', 'uneducated', 'aggressive', 'dangerous', 'stranger', 'unprofessional','unskilled','underprivileged' ]

print(p_value_exhust(A, B, E, F, 'aft'))


# Experiment 3(white status-minority status)

G = ['citizen', 'patriot', 'hero', 'captain', 'chief', 'president', 'governor','champion']
H = ['drug', 'dealer', 'homeless', 'illegal', 'immigrant', 'prostitute', 'slave', 'robery']

print(p_value_exhust(A, B, G, H, 'aft'))


100%|██████████| 12870/12870 [00:00<00:00, 140908.28it/s]


num of samples 12870
0.5353535353535354


100%|██████████| 12870/12870 [00:00<00:00, 131675.96it/s]

num of samples 12870
0.5156177156177156
num of samples 12870
0.4404040404040404
